In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import optuna
import joblib

In [15]:
# Cargar datos
sellin = pd.read_csv("datasets/sell-in.csv", sep='\t')
productos = pd.read_csv("datasets/tb_productos.csv", sep='\t')
stocks = pd.read_csv("datasets/tb_stocks.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')

# Merge inicial
df = sellin.merge(productos, on="product_id", how="left").merge(stocks, on=["product_id", "periodo"], how="left")
df['periodo_dt'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')

In [16]:
# Feature engineering
vida_producto = df.groupby("product_id")["periodo_dt"].agg(["min", "max"]).reset_index()
periodos_producto = [(p, row["product_id"]) for _, row in vida_producto.iterrows() 
                     for p in pd.date_range(row["min"], row["max"], freq="MS")]
df_pp = pd.DataFrame(periodos_producto, columns=["periodo_dt", "product_id"])
df_pp['periodo'] = df_pp['periodo_dt'].dt.strftime('%Y%m').astype(int)
df_pp = df_pp.merge(productos, on="product_id", how="left").merge(stocks, on=["product_id", "periodo"], how="left")

ventas = df.groupby(['periodo', 'product_id']).agg({'tn':'sum'}).reset_index()
df_pp = df_pp.merge(ventas, on=['periodo', 'product_id'], how='left')
df_pp['tn'] = df_pp['tn'].fillna(0)

df_pp['periodo_dt'] = pd.to_datetime(df_pp['periodo'].astype(str), format='%Y%m')
df_pp['month'] = df_pp['periodo_dt'].dt.month
df_pp['quarter'] = df_pp['periodo_dt'].dt.quarter
df_pp['year'] = df_pp['periodo_dt'].dt.year
df_pp['stock_ratio'] = df_pp.apply(lambda x: x['tn']/x['stock_final'] if x['stock_final']>0 else 0, axis=1)
df_pp['stock_ratio'] = df_pp['stock_ratio'].replace([np.inf, -np.inf], 0).fillna(0)

df_pp = df_pp.sort_values(['product_id','periodo_dt'])
for lag in range(1,4):
    df_pp[f'tn_lag_{lag}'] = df_pp.groupby('product_id')['tn'].shift(lag)
df_pp['tn_diff'] = df_pp['tn'] - df_pp['tn_lag_1']

df_pp['tn_target'] = df_pp.groupby('product_id')['tn'].shift(-2)
train_data = df_pp[~df_pp['periodo'].isin([201911,201912])]
test_data = df_pp[df_pp['periodo'] == 201912]

In [29]:
df_pp.shape

(31522, 19)

In [26]:
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import optuna
import numpy as np
import pandas as pd

# Suponemos que df_pp ya está cargado con features y target
X = df_pp.drop(columns=['periodo_dt', 'tn_target', 'tn'], errors='ignore')
y = df_pp['tn_target']

# Identificar columnas categóricas
cols_categoricas = X.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"Columnas categóricas: {cols_categoricas}")

# Codificar columnas categóricas automáticamente
for col in cols_categoricas:
    X[col] = X[col].astype('category').cat.codes



if y.isnull().any():
    print(f"⚠️ Target tiene {y.isnull().sum()} NaN. Eliminando esas filas.")
    mask = ~y.isnull()
    X = X[mask]
    y = y[mask]


# Verificar tamaño final
if X.empty or y.empty:
    print("❌ X o y están vacíos después de la limpieza. Revisa tu dataset.")
else:
    print(f"✅ Datos listos: X shape={X.shape}, y shape={y.shape}")

    # Optuna + TimeSeriesSplit
    def objective(trial):
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'num_leaves': trial.suggest_int('num_leaves', 20, 150),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True)
        }

        tscv = TimeSeriesSplit(n_splits=3)
        rmses = []
        for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            if X_train.empty or X_val.empty:
                print(f"⚠️ Fold {fold+1}: conjunto vacío. Saltando.")
                continue

            model = lgb.LGBMRegressor(**params)
            model.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
            )
            preds = model.predict(X_val)
            rmse = mean_squared_error(y_val, preds, squared=False)
            rmses.append(rmse)

        if len(rmses) == 0:
            return np.inf
        return np.mean(rmses)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)
    print("✅ Mejores parámetros:", study.best_params)

    # Entrenamiento final
    final_model = lgb.LGBMRegressor(**study.best_params)
    final_model.fit(X, y)
    print("✅ Modelo entrenado con los mejores parámetros.")


[I 2025-05-30 12:26:41,977] A new study created in memory with name: no-name-ed76e028-8ade-44b2-ab1d-67bbce3691e8


Columnas categóricas: ['cat1', 'cat2', 'cat3', 'brand']
✅ Datos listos: X shape=(31522, 16), y shape=(31522,)
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000310 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1942
[LightGBM] [Info] Number of data points in the train set: 7882, number of used features: 16
[LightGBM] [Info] Start training from score 134.240064
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000405 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1978
[LightGBM] [Info] Number of data points in the train set: 15762, number of used features: 16
[LightGBM] [Info] Start training from score 75.331341
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-05-30 12:28:11,972] Trial 49 finished with value: 7.496030586026488 and parameters: {'num_leaves': 37, 'max_depth': 5, 'learning_rate': 0.04485811502729954, 'n_estimators': 634, 'subsample': 0.8759515711439879, 'colsample_bytree': 0.8656833410912196, 'reg_alpha': 2.0419205412424155e-06, 'reg_lambda': 1.7657669987870907e-07}. Best is trial 41 with value: 6.518257139179563.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[552]	valid_0's rmse: 6.35989
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000770 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2028
[LightGBM] [Info] Number of data points in the train set: 23642,

In [21]:
print("Mejores parámetros encontrados:", study.best_params)

Mejores parámetros encontrados: {'num_leaves': 144, 'max_depth': 15, 'learning_rate': 0.07766119322368795, 'n_estimators': 728, 'subsample': 0.6188111024172903, 'colsample_bytree': 0.998780906695816, 'reg_alpha': 0.0029833791412988433, 'reg_lambda': 5.520787178879811e-07}


In [27]:
# 🏁 Entrenamiento final con los mejores hiperparámetros
final_model = lgb.LGBMRegressor(**study.best_params)
final_model.fit(X, y)
print("✅ Modelo final entrenado.")

# 📅 Preparar conjunto de prueba
X_test = test_data.drop(columns=['periodo_dt','tn_target','tn'], errors='ignore')

# Codificar columnas categóricas igual que en entrenamiento
cols_categoricas_test = X_test.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"Columnas categóricas en test: {cols_categoricas_test}")

for col in cols_categoricas_test:
    X_test[col] = X_test[col].astype('category').cat.codes

# Convertir a float32 para consistencia
X_test = X_test.astype(np.float32)

# Filtrar solo productos a predecir
productos_a_predecir = pd.read_csv("datasets/product_id_apredecir201912.csv")
X_test = X_test[X_test['product_id'].isin(productos_a_predecir['product_id'])]

# Verificar que no esté vacío
if X_test.empty:
    print("❌ X_test está vacío después del filtrado. Revisa los datos.")
else:
    # 🔮 Hacer predicciones
    preds = final_model.predict(X_test)

    # 📝 Crear DataFrame con resultados
    result = pd.DataFrame({
        "product_id": X_test["product_id"].values,
        "tn": preds
    })

    # 💾 Guardar en CSV
    result.to_csv("./kaggle/optimized_lgb.csv", index=False)
    print("✅ Predicciones guardadas en ./kaggle/optimized_lgb.csv")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000213 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2072
[LightGBM] [Info] Number of data points in the train set: 31522, number of used features: 16
[LightGBM] [Info] Start training from score 39.421567
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [28]:
result['product_id'] = result['product_id'].astype(int)
result.to_csv("./kaggle/optimized_lgb.csv", index=False)
result

,product_id,tn
0,20001,21.206227
1,20002,22.163006
2,20003,12.595877
3,20004,22.879437
4,20005,-5.844806
...,...,...
775,21263,-0.141418
776,21265,-0.044921
777,21266,-0.044921
778,21267,-0.045546
